In [1]:
import torch  # Импорт основного модуля PyTorch
from torchviz import make_dot  # Импорт функции для визуализации графа вычислений
import torch.nn as nn  # Импорт модуля нейронных сетей PyTorch
from torch.nn import functional as F  # Импорт функционального модуля PyTorch
import mmap  # Импорт модуля для работы с памятью
import random  # Импорт модуля для генерации случайных чисел
import pickle  # Импорт модуля для сериализации данных
import os  # Импорт модуля для работы с операционной системой
import datetime  # Импорт модуля для работы с датой и временем
import matplotlib.pyplot as plt

# Определение устройства для выполнения вычислений: GPU, если доступно, иначе CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

CUDA_LAUNCH_BLOCKING=1

# Определение параметров для обучения модели
batch_size = 64  # Размер батча
block_size = 128  # Размер блока
max_iters = 3000  # Максимальное количество итераций
learning_rate = 1e-3  # Скорость обучения
eval_iters = 100  # Количество итераций для оценки
n_embd = 384  # Размерность вектора вложений
n_head = 1  # Количество голов в механизме внимания
n_layer = 1  # Количество слоёв в модели + слоёв rnn
dropout = 0.2  # Вероятность отключения нейронов

# Вывод используемого устройства
print(device)


cuda


In [2]:
chars = ""  # Инициализация строки для хранения уникальных символов

# Открытие словая в режиме чтения с указанием кодировки utf-8
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()  # Чтение содержимого файла в строку
    chars = sorted(list(set(text)))  # Получение отсортированного списка уникальных символов
    print(chars)  # Вывод списка уникальных символов

vocab_size = len(chars)  # Вычисление размера словаря (количества уникальных символов)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
# Создание словаря для преобразования символов в целые числа
string_to_int = { ch:i for i,ch in enumerate(chars) }

# Создание словаря для преобразования целых чисел обратно в символы
int_to_string = { i:ch for i,ch in enumerate(chars) }

# Функция для кодирования строки в список индексов символов
encode = lambda s: [string_to_int[c] for c in s]

# Функция для декодирования списка индексов обратно в строку
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [4]:
def get_random_chunk(split):
    # Выбор имени файла в зависимости от типа набора данных: обучающего или валидационного
    filename = "wizard_of_oz.txt" if split == 'train' else "wizard_of_oz.txt"
    
    # Открытие файла в двоичном режиме
    with open(filename, 'rb') as f:
        # Создание объекта памяти с доступом только для чтения
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            file_size = len(mm)  # Определение размера файла
            # Генерация случайной начальной позиции для чтения данных
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            mm.seek(start_pos)  # Перемещение указателя на начальную позицию
            # Чтение блока данных размером block_size*batch_size-1
            block = mm.read(block_size*batch_size-1)

            # Декодирование блока данных и удаление символов возврата каретки
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')

            # Кодирование декодированного блока в тензор PyTorch
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data  # Возвращение тензора данных

def get_batch(split):
    data = get_random_chunk(split)  # Получение случайного фрагмента данных
    # Генерация случайных индексов для создания батча
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # Создание батча входных данных
    x = torch.stack([data[i:i+block_size] for i in ix])
    # Создание батча целевых данных (сдвиг на один символ вперёд)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # Перемещение батчей на устройство для вычислений
    x, y = x.to(device), y.to(device)
    return x, y  # Возвращение батчей входных и целевых данных

In [5]:
class RNNLayer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(RNNLayer, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True)

    def forward(self, x):
        outputs, _ = self.rnn(x)
        return outputs

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        # Инициализация слоев для ключа, запроса и значения без использования смещения
        self.key = nn.Linear(n_embd, head_size, bias=False)  # Слой для ключей
        self.query = nn.Linear(n_embd, head_size, bias=False)  # Слой для запросов
        self.value = nn.Linear(n_embd, head_size, bias=False)  # Слой для значений

        # Создание маски для последующего применения в операции внимания
        # Маска позволяет модели учитывать только предыдущие и текущие токены
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
        # Слой dropout для регуляризации
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape  # Размеры входного тензора: batch, sequence length, channels
        k = self.key(x)  # Преобразование входного тензора в ключи
        q = self.query(x)  # Преобразование входного тензора в запросы
        # Расчет весов внимания с использованием скалярного произведения запросов и ключей
        # и масштабирование на корень квадратный из размерности ключей
        wei = q @ k.transpose(-2, -1) * (k.shape[-1] ** -0.5)
        
        # Применение маски для обеспечения каузальности внимания
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        
        # Применение softmax для нормализации весов внимания
        wei = F.softmax(wei, dim=-1)
        
        # Применение dropout к весам внимания
        wei = self.dropout(wei)
        
        v = self.value(x)  # Преобразование входного тензора в значения
        out = wei @ v  # Вычисление выхода с использованием весов внимания и значений
        
        return out  # Возвращение результата

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        # Инициализация списка модулей внимания для каждой головы
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        # Линейный слой для проекции конкатенированных выходов всех голов на исходное пространство вложений
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        # Слой dropout для регуляризации
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Конкатенация выходов всех голов внимания вдоль последнего измерения
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        # Применение линейной проекции и dropout к конкатенированному выходу
        out = self.dropout(self.proj(out))
        return out  # Возвращение результата   

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        # Определение последовательной нейронной сети с двумя полносвязными слоями и функцией активации
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),  # Первый полносвязный слой увеличивает размерность
            nn.ReLU(),  # Функция активации ReLU
            nn.Linear(4 * n_embd, n_embd),  # Второй полносвязный слой возвращает размерность
            nn.Dropout(dropout),  # Слой dropout для регуляризации
        )

    def forward(self, x):
        return self.net(x)  # Пропускание входного тензора через определённую сеть

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        # Вычисление размера каждой головы внимания
        head_size = n_embd // n_head
        # Инициализация многоголового внимания и полносвязной сети
        self.sa = MultiHeadAttention(n_head, head_size)  # Многоголовое внимание
        self.ffwd = FeedFoward(n_embd)  # Полносвязная сеть
        # Инициализация слоев нормализации
        self.ln1 = nn.LayerNorm(n_embd)  # Нормализация после многоголового внимания
        self.ln2 = nn.LayerNorm(n_embd)  # Нормализация после полносвязной сети

    def forward(self, x):
        # Применение механизма многоголового внимания
        y = self.sa(x)
        # Сложение выхода внимания с входным тензором и нормализация
        x = self.ln1(x + y)
        # Применение полносвязной сети
        y = self.ffwd(x)
        # Сложение выхода полносвязной сети с результатом первой нормализации и нормализация
        x = self.ln2(x + y)
        return x  # Возвращение результата

    
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Инициализация встраивания токенов и позиций, блоков трансформера, нормализации и выходного слоя
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)  # Встраивание токенов
        self.position_embedding_table = nn.Embedding(block_size, n_embd)  # Встраивание позиций
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])  # Блоки трансформера
        self.ln_f = nn.LayerNorm(n_embd)  # Слой нормализации
        self.lm_head = nn.Linear(n_embd, vocab_size)  # Выходной линейный слой

        # Добавление RNN слоя
        self.rnn_layer = RNNLayer(n_embd, n_embd, n_layer, dropout)
        
        # Инициализация весов модели
        self.apply(self._init_weights)

    def _init_weights(self, module):
        # Инициализация весов линейных и встраивающих слоев
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        # Получение размеров входного индекса
        B, T = index.shape
        # Получение векторных представлений для токенов и позиций
        tok_emb = self.token_embedding_table(index)  # Встраивание токенов
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # Встраивание позиций
        x = tok_emb + pos_emb  # Сложение встраиваний
        
        # Применение RNN слоя
        x = self.rnn_layer(x)

        # Применение блоков трансформера
        x = self.blocks(x) 
        x = self.ln_f(x)  # Применение нормализации
        logits = self.lm_head(x)  # Получение логитов
        
        # Вычисление потерь, если цели предоставлены
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None
            
        return logits, loss  # Возвращение логитов и потерь
    
    def generate(self, index, max_new_tokens):
        # Генерация новых токенов
        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]  # Использование последних `block_size` токенов
            logits, loss = self.forward(index_cond)  # Прямой проход
            logits = logits[:, -1, :]  # Выбор последнего столбца логитов
            probs = F.softmax(logits, dim=-1)  # Применение софтмакса
            index_next = torch.multinomial(probs, num_samples=1)  # Сэмплирование следующего токена
            index = torch.cat((index, index_next), dim=1)  # Обновление индекса
        
        return index  # Возвращение обновленного индекса


model = GPTLanguageModel(vocab_size)  # Инициализация модели GPT с размером словаря
print('Загрузка параметров...')
if os.path.exists('model_rnn-01.pkl'):
    with open('model-01.pkl', 'rb') as f:
        model = pickle.load(f)  # Загрузка модели из файла
    print('Загрузка выполнена')
else:
    print('Файл не найден. Будет создан новый файл.')  # Сообщение, если файл не найден
m = model.to(device)  # Перемещение модели на устройство для вычислений



Загрузка параметров...


Файл не найден. Будет создан новый файл.


In [6]:
input_tensor = torch.randint(0, vocab_size, (128, block_size)).to(device)  # Создание тензора случайных индексов

outputs = model(input_tensor)  # Прямой проход модели

output_tensor = outputs[0]  # Получение выходного тензора

dummy_target = torch.randint(0, vocab_size, output_tensor.shape).to(device)  # Создание тензора целей

# Создание визуализации графа вычислений
dot = make_dot((output_tensor, dummy_target), params=dict(model.named_parameters()))

current_date = datetime.datetime.now().strftime("%y-%m-%d_%H-%M-%S")  # Получение текущей даты и времени
file_name = f"m_model{current_date}"  # Формирование имени файла

# Сохранение визуализации в формате SVG с добавлением 'svg' в имя файла
dot.render(f"gpt\\{file_name}_svg", format="svg")

# Сохранение визуализации в формате JPG с добавлением 'jpg' в имя файла
dot.render(f"gpt\\{file_name}_jpg", format="jpg")

'gpt\\m_model24-03-20_23-25-54_jpg.jpg'

In [7]:
@torch.no_grad()  # Декоратор для отключения градиентов (для уменьшения использования памяти и ускорения вычислений)
def estimate_loss():
    out = {}  # Словарь для хранения результатов
    model.eval()  # Перевод модели в режим оценки (выключение dropout и т.д.)
    for split in ['train', 'val']:  # Итерация по обучающему и валидационному наборам данных
        losses = torch.zeros(eval_iters)  # Тензор для хранения потерь на каждой итерации
        for k in range(eval_iters):  # Итерация для оценки средней потери
            X, Y = get_batch(split)  # Получение батча данных
            logits, loss = model(X, Y)  # Прямой проход модели и вычисление потери
            losses[k] = loss.item()  # Сохранение значения потери
        out[split] = losses.mean()  # Вычисление средней потери для набора данных
    model.train()  # Перевод модели обратно в режим обучения
    return out  # Возвращение словаря с результатами


In [8]:
@torch.no_grad()  # Декоратор для отключения градиентов
def estimate_perplexity():
    out = {}  # Словарь для хранения результатов
    model.eval()  # Перевод модели в режим оценки
    for split in ['train', 'val']:  # Итерация по обучающему и валидационному наборам данных
        perplexities = torch.zeros(eval_iters)  # Тензор для хранения перплексий
        for k in range(eval_iters):  # Итерация для оценки средней перплексии
            X, Y = get_batch(split)  # Получение батча данных
            logits, loss = model(X, Y)  # Прямой проход модели и вычисление потери
            perplexity = torch.exp(loss).item()  # Вычисление перплексии
            perplexities[k] = perplexity  # Сохранение значения перплексии
        out[split] = perplexities.mean()  # Вычисление средней перплексии для набора данных
    model.train()  # Перевод модели обратно в режим обучения
    return out  # Возвращение словаря с результатами


In [9]:
# Создание оптимизатора PyTorch AdamW с заданной скоростью обучения
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Цикл обучения модели
for iter in range(max_iters):
    #print(iter)
    # Оценка потерь и перплексии на каждой eval_iters итерации
    if iter % eval_iters == 0:
        losses = estimate_loss()  # Оценка потерь
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

        perplexitys = estimate_perplexity()  # Оценка перплексии
        print(f"step: {iter}, train perplexity: {perplexitys['train']:.3f}, val perplexity {perplexitys['val']:.3f}")
    
    # Получение батча данных для обучения
    xb, yb = get_batch('train')

    # Вычисление потерь
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)  # Обнуление градиентов
    loss.backward()  # Вычисление градиентов
    optimizer.step()  # Обновление весов модели

# Вывод последней потери и перплексии после обучения
print(loss.item())
perplexity = torch.exp(loss)
print(perplexity.item())

# Сохранение модели в файл
with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('модель сохранена')


step: 0, train loss: 4.473, val loss: 4.472
step: 0, train perplexity: 87.816, val perplexity 87.887
step: 100, train loss: 3.157, val loss: 3.144
step: 100, train perplexity: 23.228, val perplexity 23.263
step: 200, train loss: 3.136, val loss: 3.135
step: 200, train perplexity: 22.908, val perplexity 22.879
step: 300, train loss: 3.151, val loss: 3.146
step: 300, train perplexity: 23.129, val perplexity 23.145
step: 400, train loss: 3.142, val loss: 3.138
step: 400, train perplexity: 23.076, val perplexity 23.292
step: 500, train loss: 3.135, val loss: 3.133
step: 500, train perplexity: 22.994, val perplexity 22.884
step: 600, train loss: 3.146, val loss: 3.142
step: 600, train perplexity: 23.072, val perplexity 23.066
step: 700, train loss: 3.140, val loss: 3.146
step: 700, train perplexity: 23.170, val perplexity 23.183
step: 800, train loss: 3.128, val loss: 3.135
step: 800, train perplexity: 23.223, val perplexity 22.971
step: 900, train loss: 3.138, val loss: 3.137
step: 900, tr

In [10]:
# Измененный prompt для генерации текста
prompt = 'D'

# Кодирование prompt в тензор индексов для использования в модели
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)

# Генерация новых символов с помощью модели
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())

# Вывод сгенерированного текста
print(generated_chars)


Dhel trvdn f n"eaJdoedwwtnusw eebdim  h irrorv  sW neetmpnneeauesfsoo
pr
ofm y d ,tcs
la n hcrtthohw 
